In [58]:
import mlflow
from mlflow.exceptions import RestException
import pandas as pd
from utils import common_functions 
import numpy as np

import plotly.graph_objects as go

In [14]:
remote_server_uri = "http://34.58.215.162:8080/"  # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)

#Creacion o identificacion del experimento 
try:
    experiment_name = 'INV_WV_PLUS_MODELS' #Puede ser cualquiera siempre y cuando no se troque con otro
    experiment_id = (mlflow
                        .create_experiment(name=experiment_name
                                            ,tags={'created_by':'Victor Moreno'})) #importante poner el nombre de quien lo crea
except RestException as r:
    print(r)
    experiment = mlflow.get_experiment_by_name(experiment_name)
    print('Full name',experiment.name)
    experiment_id = experiment.experiment_id

RESOURCE_ALREADY_EXISTS: Experiment 'INV_WV_PLUS_MODELS' already exists.
Full name INV_WV_PLUS_MODELS


In [15]:

logged_model = 'runs:/fc82ffafa2e74ae389d444f943fe8d60/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
m = loaded_model.get_raw_model()
m.to_json()



/Users/victormoreno/Desktop/master/TFE/Develop/tfm_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'{"module": "keras", "class_name": "Sequential", "config": {"name": "sequential", "trainable": true, "dtype": {"module": "keras", "class_name": "DTypePolicy", "config": {"name": "float32"}, "registered_name": null}, "layers": [{"module": "keras.layers", "class_name": "InputLayer", "config": {"batch_shape": [null, 24, 2], "dtype": "float32", "sparse": false, "name": "input_layer"}, "registered_name": null}, {"module": "keras.layers", "class_name": "LSTM", "config": {"name": "lstm", "trainable": true, "dtype": {"module": "keras", "class_name": "DTypePolicy", "config": {"name": "float32"}, "registered_name": null}, "return_sequences": true, "return_state": false, "go_backwards": false, "stateful": false, "unroll": false, "zero_output_for_mask": false, "units": 100, "activation": "relu", "recurrent_activation": "sigmoid", "use_bias": true, "kernel_initializer": {"module": "keras.initializers", "class_name": "GlorotUniform", "config": {"seed": null}, "registered_name": null}, "recurrent_ini

In [21]:
df = pd.read_csv('data/datos_PEPEUSDT.csv',header=0)

#multipling by 1M the close data
df['closex1M'] = df['Close']*1000000 #PEPE

select = ['Close time_date','closex1M']
df_clean = df[select]

In [22]:
get_wt_coeff_inv = common_functions().get_wt_coeff_inv
plot_inv_wv = common_functions().plot_inv_wv
create_sequences = common_functions().create_sequences
standarize_data = False

In [24]:
n = df_clean.shape[0] #Cantidad de puntos a tratar
data = np.array(df_clean['closex1M'][:n]) #valores de la serie temporal
dates = df_clean['Close time_date'][:n] #valores de las fechas

#llamando la funcion de get_wt_coeff_inv para obtener 
## Coeficientes de wavelet y la senal resconstruida desde estos
coeffs_lv3, inv_coeffs_lv3 = get_wt_coeff_inv(signal=data
                                      ,wavelet='db1'
                                      ,level=3
                                      ,mode='symmetric'
                                      ,take=n)

coeffs_lv1, inv_coeffs_lv1 = get_wt_coeff_inv(signal=data
                                      ,wavelet='db1'
                                      ,level=1
                                      ,mode='symmetric'
                                      ,take=n)

In [25]:
aproximacion = 'cA3'
detalle = 'cD1'
df_train = pd.DataFrame({aproximacion:inv_coeffs_lv3[aproximacion]
                         ,detalle:inv_coeffs_lv1[detalle]})
window_size = 24


X, Y = create_sequences(df_train,window_size=window_size,target_col=aproximacion)

In [145]:
test_window = 48
history = 24
last_pos = X.shape[0]- test_window 
print(last_pos)
last_windows = X[last_pos-history:last_pos+1]

15273


In [143]:
import numpy as np

def predict_future(model, last_known_data, num_predictions=48):
    """
    Predicts the next `num_predictions` steps recursively using the trained LSTM model.

    Parameters:
    - model: Trained LSTM model.
    - last_known_data: The last available input data (should match the input shape).
    - num_predictions: Number of future time steps to predict.

    Returns:
    - future_predictions: List of predicted values.
    """
    future_predictions = []
    current_input = last_known_data.copy()  # Copy to avoid modifying original data

    for _ in range(num_predictions):
        # Ensure input shape matches model expectation
        current_input_reshaped = current_input.reshape(1, current_input.shape[0], current_input.shape[1])  
        
        # Predict next time step
        next_prediction = model.predict(current_input_reshaped, verbose=0)

        # Store prediction
        future_predictions.append(next_prediction[0, 0])  # Assuming single output per time step

        # Update input: Remove oldest value and add the new prediction
        current_input = np.roll(current_input, shift=-1, axis=0)  # Shift left
        current_input[-1, 0] = next_prediction  # Replace last entry with prediction

    return np.array(future_predictions)

# Example usage
last_window = X[last_pos]  # Last available input window
future_24h_predictions = predict_future(m, last_window, num_predictions=test_window)

#print(future_24h_predictions)


/var/folders/xq/wt557f817vv282xt9gj_2bzh0000gn/T/ipykernel_2689/3767544082.py:30: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



In [146]:
import numpy as np

def predict_future(model, past_windows, num_predictions=24):
    """
    Predicts the next `num_predictions` steps recursively using multiple previous windows.

    Parameters:
    - model: Trained LSTM model.
    - past_windows: A list of past input windows (each should match the model's input shape).
    - num_predictions: Number of future time steps to predict.

    Returns:
    - future_predictions: List of predicted values.
    """
    future_predictions = []
    num_windows = len(past_windows)

    for _ in range(num_predictions):
        window_predictions = []

        # Make predictions for each of the past windows
        for window in past_windows:
            current_input_reshaped = window.reshape(1, window.shape[0], window.shape[1])  
            next_prediction = model.predict(current_input_reshaped, verbose=0)
            window_predictions.append(next_prediction[0, 0])  # Assuming single output per step

        # Average predictions from multiple past windows
        avg_prediction = np.mean(window_predictions)
        future_predictions.append(avg_prediction)

        # Update input windows: Shift left and add the new prediction
        for i in range(num_windows):
            past_windows[i] = np.roll(past_windows[i], shift=-1, axis=0)  # Shift left
            past_windows[i][-1, 0] = avg_prediction  # Update last entry

    return np.array(future_predictions)

# Example usage
#num_past_windows = 4  # Use last 4 sliding windows
#past_windows = [X[-(i+1)] for i in range(num_past_windows)]  # Get last 4 windows

future_24h_predictions = predict_future(m, last_windows, num_predictions=test_window)


In [149]:
future_24h_predictions

array([12.660961 , 12.637245 , 12.613716 , 12.590297 , 12.564802 ,
       12.542342 , 12.516487 , 12.491956 , 12.464635 , 12.439692 ,
       12.413494 , 12.384705 , 12.357765 , 12.331845 , 12.303805 ,
       12.275902 , 12.248585 , 12.2219715, 12.195254 , 12.168297 ,
       12.141078 , 12.112842 , 12.085387 , 12.060156 , 12.034435 ,
       12.008039 , 11.982455 , 11.95761  , 11.93137  , 11.908757 ,
       11.883431 , 11.860058 , 11.834497 , 11.811807 , 11.788348 ,
       11.76281  , 11.73948  , 11.717482 , 11.693752 , 11.670483 ,
       11.6480465, 11.626551 , 11.605191 , 11.583843 , 11.5624695,
       11.540325 , 11.519144 , 11.500297 ], dtype=float32)

In [150]:
pos_train = df_train.shape[0] - test_window# - history
pos_train

15297

In [151]:
fig = go.Figure()

date_pred_24h = dates[pos_train:]

fig.add_trace(go.Scatter(x=dates
                            ,y=data
                            ,mode='lines'
                            ,name='real'
                            ,line = dict(color='green')))


fig.add_trace(go.Scatter(x=date_pred_24h
                            ,y=future_24h_predictions
                            ,mode='lines'
                            ,name='test'
                            ,line= dict(color='red')))


    
fig.show()

